<a href="https://colab.research.google.com/github/Autobot37/jupyter/blob/main/WengertAutograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [ ]:
x = torch.tensor([2.0],requires_grad=True)


In [ ]:
y = x**2


In [ ]:
y.grad,x.grad

<ipython-input-8-bde24807d4bc>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:486.)
  y.grad,x.grad


(None, None)

In [ ]:
torch.autograd.grad(y,x)

(tensor([4.]),)

In [ ]:
y.backward()

In [ ]:
x = torch.tensor([2.0], requires_grad=True)
y = torch.tensor([3.0], requires_grad=True)
z = x * y
w = x+z

In [ ]:
grads = torch.autograd.grad(outputs=(z,w), inputs=(x, y))

# Print the gradients
print(grads)

(tensor([7.]), tensor([4.]))


In [35]:
G = {
    "add": lambda a, b: int(a) + int(b),
    "square": lambda a: int(a) ** 2
}


In [36]:
x1 = 3
x2 = 5

In [134]:
F = [
    ("z1","add",("x1","x1")),
    ("z2","add",("z1","x2")),
    ("f","square",("z2",))
]
val = {"x1":3, "x2":5}

In [135]:
#f = (2 x1 + x2)^2

In [136]:
def eval(f):
  for z,g,args in f:
    op = G[g]
    if isinstance(args,tuple):
      val[z] = op(*[val[arg] for arg in args])
    
    else:
      val[z] = op(val[args])
  return val

In [137]:
eval(F)

{'x1': 3, 'x2': 5, 'z1': 6, 'z2': 11, 'f': 121}

In [138]:
DG = { "add" : [ (lambda a,b: 1), (lambda a,b: 1) ],
"square": [ lambda a:2*a ] }

In [139]:
DG["square"][0]

<function __main__.<lambda>(a)>

In [140]:
reversed(val)

In [141]:
for a,b,args in reversed(F):
  print(a,b,args)
  for i in range(len(args)):
    print(i)

f square ('z2',)
0
z2 add ('z1', 'x2')
0
1
z1 add ('x1', 'x1')
0
1


In [142]:
F[-1][-1]

('z2',)

In [145]:
def backprop(f, val):
    delta = {}
    delta["f"] = 1 
    for z, g, args in reversed(f):
        for i in range(len(args)):
            op = DG[g][i]
            if args[i] not in delta:
                delta[args[i]] = 0
            if isinstance(args, tuple):
                delta[args[i]] += delta[z] * op(*[val[arg] for arg in args])
            else:
                delta[args[i]] += delta[z] * op(val[args])
    return delta


In [146]:
backprop(F,val)

{'f': 1, 'z2': 22, 'z1': 22, 'x2': 22, 'x1': 44}

In [147]:
##constructing wengert lists